In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [4]:
base_html = 'C://Users//bruno//Documents//DS//DataSets Grandes//Historicos Economicos//BLUE Historico.html'

Fuente: https://www.ambito.com/contenidos/dolar-informal-historico.html

In [5]:
blue_hist = pd.read_html(base_html) # this parses all the tables in webpages to a list
df_blue = blue_hist[0]
df_blue.head() # Values are int without decimals, which is incorrect due to the real value's nature

,Fecha,Compra,Venta
0,08-07-2021,16900,17400
1,07-07-2021,16700,17200
2,06-07-2021,16600,17100
3,05-07-2021,16600,17100
4,02-07-2021,16500,17000


In [ ]:
new_value_list = []
for value in df_blue.Compra:
    ss1, ss2 = str(value)[:2], str(value)[2:] # Slicing the value as string to get separated the number
    new = ss1+'.'+ss2 # Adding the . into the value
    new_value_list.append(float(new)) # Adding the new value to a list}
df_.Referencia = pd.Series(new_value_list) # Converting it to a Series and inserting replacing the extracted values

In [ ]:
def decimals(df, column):
    '''Converts integer value into float considering last 2 numbers as decimals. df parameter to specify df, column parameter to specify target column of df'''
    new_value_list = []
    for value in df_blue.Compra:
        ss1, ss2 = str(value)[:2], str(value)[2:] # Slicing the value as string to get separated the number
        new = ss1+'.'+ss2 # Adding the . into the value
        new_value_list.append(float(new)) # Adding the new value to a list}
    df_.columna = pd.Series(new_value_list) # Converting it to a Series and inserting replacing the extracted values